<a href="https://colab.research.google.com/github/JimenaBaripatti/FeatureEngineering/blob/main/Fire_FSA_and_demographic_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Fire data

In [ ]:
# setting up libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm 
import statistics
from sklearn.metrics import matthews_corrcoef
from scipy.stats import chi2_contingency
import math
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import IsolationForest
from scipy.stats import zscore

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

%matplotlib inline


# Setting working directory
os.chdir("/content/drive/MyDrive/Colab Notebooks/Data/")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
df=pd.read_csv('Fire Incidents Data.csv')

##Add new feature: FSA based on Latitude and Longitude

In [ ]:
#df['latitude']=str(df['latitude'])
#df['latitude'].dtypes

KeyError: ignored

In [ ]:
#df['Coord']=df[['Latitude','Longitude']].apply(lambda x: ','.join(x.astype(str)), axis=1)

In [ ]:
#Extract postal code from latitude and longitude information 
#Reverse Geocoding

from geopy.geocoders import Nominatim #from open street maps and its free
from geopy.geocoders import GoogleV3
from geopy.geocoders import Bing

# create the locator
geolocator = Nominatim(user_agent="jsqzhu@gmail.com") # user_agent="specify_your_app_name_here" 
# you may need to use your own email if the next step gives you HTTP Error 403


Dixon Road, West Humber-Clairville, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9W 5N4, Canada


In [ ]:
# getting the postal code
df['location'] = df.apply(lambda x: geolocator.reverse(str(x['Latitude'])+", "+str(x['Longitude'])).address.split(",")[-2], axis=1)

KeyboardInterrupt: ignored

In [ ]:
# extracting FSA from location: the first three letters from 'location'

KeyboardInterrupt: ignored

##Explore data

In [ ]:
df['Area_of_Origin'].value_counts()

24 - Cooking Area or Kitchen                                                 2878
81 - Engine Area                                                             1931
44 - Trash, Rubbish Storage (inc garbage chute room, garbage/industri        1070
64 - Porch or Balcony                                                        957 
22 - Sleeping Area or Bedroom (inc. patients room, dormitory, etc)           871 
21 - Living Area (e.g. living, TV, recreation, etc)                          596 
75 - Trash, rubbish area (outside)                                           429 
97 - Other - unclassified                                                    418 
99 - Undetermined  (formerly 98)                                             402 
42 - Garage                                                                  401 
83 - Electrical Systems                                                      396 
27 - Laundry Area                                                            366 
25 - Washroom or

In [ ]:
df['Final_Incident_Type'].value_counts()

01 - Fire                                                                                  15516
03 - NO LOSS OUTDOOR fire (exc: Sus.arson,vandal,child playing,recycling or dump fires)    1914 
02 - Explosion (including during Fire, excluding Codes 3 & 11-13)                          106  
Name: Final_Incident_Type, dtype: int64

In [ ]:
df['Initial_CAD_Event_Type'].value_counts()

FIR                                       3929
Fire - Grass/Rubbish                      1698
VEF                                       1652
FIHR                                      1617
FICI                                      1303
FIG                                       917 
Fire - Residential                        898 
FAHR                                      787 
VEFH                                      548 
Vehicle Fire                              478 
Fire -  Highrise Residential              379 
Fire - Commercial/Industrial              375 
FACI                                      303 
Alarm Highrise Residential                235 
FIHRD                                     199 
FAR                                       175 
Vehicle Fire - Highway                    157 
FAHRD                                     154 
FITP                                      113 
FIS                                       107 
CC                                        82  
Alarm Commerc

In [ ]:
df['Property_Use'].value_counts(normalize=True)

323 - Multi-Unit Dwelling - Over 12 Units                                        0.220644
301 - Detached Dwelling                                                          0.141774
901 - Automobile                                                                 0.082350
896 - Sidewalk, street, roadway, highway, hwy (do not use for fire incidents)    0.053721
302 - Semi-Detached Dwelling                                                     0.044140
848 - Trash/Rubbish/Garbage Container or Dumpster                                0.042315
303 - Attached Dwelling (eg. rowhouse, townhouse, etc.)                          0.034502
861 - Open Land (eg. light ground cover, bush, grass, etc.)                      0.027032
902 - Small Truck (eg. pick-up, van, etc.)                                       0.020816
321 - Multi-Unit Dwelling - 2 to 6 Units                                         0.016424
331 - Apartment, Flat, Tenement with Business                                    0.016196
151 - Rest

In [ ]:
df['Property_Use'].isnull().sum()

1

# Demographic data

In [ ]:
#load household/dwelling structure/construction age by FSA
df_household=pd.read_csv('2016_CensusDataFSA_HouseholdStructure.csv') #possible features: ratio of highrises vs other, age of structure??
#load poulation data by FSA
df_population=pd.read_csv('2016_CensusDataFSA_Population.csv') #population density? need to find area
#load family structure by FSA (might be redundant) 
df_family=pd.read_csv('2016_CensusDataFSA_FamilyStructure.csv')

In [ ]:
#load geographic data
fsa=pd.read_csv('fsa_area_2016.csv')
#set first row as column names
fsa.columns=fsa.iloc[0]
fsa=fsa[1:]

In [ ]:
fsa=fsa.rename(columns={"CFSAUID":"Geo Code"})

##Population density per FSA

In [ ]:
df_population['Category'].unique()

array(['Population; 2016', 'Population; 2011',
       'Population percentage change; 2011 to 2016',
       'Total private dwellings',
       'Private dwellings occupied by usual residents',
       'Population density per square kilometre',
       'Land area in square kilometres'], dtype=object)

In [ ]:
population_by_FSA=df_population[df_population['Category']=='Population; 2016']

In [ ]:
# join population and area data together
population_density_by_FSA=pd.merge(population_by_FSA[['Geo Code','Counts']], fsa, on='Geo Code', how='inner')
#calculate density
population_density_by_FSA['Counts']=pd.to_numeric(population_density_by_FSA['Counts'])
population_density_by_FSA['Area Sq Km']=pd.to_numeric(population_density_by_FSA['Area Sq Km'])
population_density_by_FSA['density']=round(population_density_by_FSA['Counts']/population_density_by_FSA['Area Sq Km'],2)
#population_density_by_FSA.dtypes

In [ ]:
population_density_by_FSA.head()

,Geo Code,Counts,PRUID,PRNAME,Area Sq Km,density
0,M1B,66108,35,Ontario,37.470902,1764.25
1,M1C,35626,35,Ontario,14.761468,2413.45
2,M1G,29690,35,Ontario,9.496478,3126.42
3,M1H,24383,35,Ontario,5.121544,4760.87
4,M1J,36699,35,Ontario,6.438187,5700.21


In [ ]:
population_density_by_FSA=population_density_by_FSA.drop(columns=[col for col in population_density_by_FSA.columns if col not in ['Geo Code','density']], axis=1)

In [ ]:
population_density_by_FSA.head()

,Geo Code,density
0,M1B,1764.25
1,M1C,2413.45
2,M1G,3126.42
3,M1H,4760.87
4,M1J,5700.21


##building condition

In [ ]:
df_household['Category'].unique()

array(['Total - Private households by household type - 100% data',
       'Total - Private households by tenure - 25% sample data',
       'Total - Occupied private dwellings by condominium status - 25% sample data',
       'Total - Occupied private dwellings by number of bedrooms - 25% sample data',
       'Total - Occupied private dwellings by number of rooms - 25% sample data',
       'Average number of rooms per dwelling',
       'Total - Private households by number of persons per room - 25% sample data',
       'Total - Private households by housing suitability - 25% sample data',
       'Total - Occupied private dwellings by period of construction - 25% sample data',
       'Total - Occupied private dwellings by dwelling condition - 25% sample data',
       'Total - Private households by number of household maintainers - 25% sample data',
       'Total - Private households by age of primary household maintainers - 25% sample data',
       'Total -  Owner and tenant households wi

In [ ]:
#residential building condition
df_condition=df_household.loc[df_household['Category']=='Total - Occupied private dwellings by dwelling condition - 25% sample data']
df_condition=df_condition.pivot(index='Geo Code', columns='Sub Category', values='Counts')

In [ ]:
#clean up data frame
df_condition=df_condition.rename(columns={df_condition.columns[0]: 'Total'})
df_condition=df_condition[df_condition['Total']!='x']
df_condition['Major repairs needed']=pd.to_numeric(df_condition['Major repairs needed'])
df_condition['Total']=pd.to_numeric(df_condition['Total'])
#proportion of residents that require major repairs
df_condition['poor_building_condition']=(df_condition['Major repairs needed'])/(df_condition['Total'])

In [ ]:
df_condition=df_condition.reset_index()
df_condition.head()

Sub Category,Geo Code,Total,Major repairs needed,Only regular maintenance or minor repairs needed,poor_building_condition
0,Canada,14072075,920480,13151600,0.065412
1,M1B,20235,1130,19105,0.055844
2,M1C,11270,465,10805,0.041260
3,M1G,9825,1075,8750,0.109415
4,M1H,8930,500,8435,0.055991


In [ ]:
df_condition=df_condition.drop(columns=[col for col in df_condition.columns if col not in ['Geo Code','poor_building_condition']], axis=1)

## dwell type: % condo

In [ ]:
#dwelling type (condo vs non-condo)
df_dwell=df_household.loc[df_household['Category']=='Total - Occupied private dwellings by condominium status - 25% sample data']
df_dwell=df_dwell.pivot(index='Geo Code', columns='Sub Category', values='Counts')

In [ ]:
df_dwell.columns

Index([nan, 'Condominium', 'Not condominium'], dtype='object', name='Sub Category')

In [ ]:
#clean up data frame
df_dwell=df_dwell.rename(columns={df_dwell.columns[0]: 'Total'})
df_dwell=df_dwell[df_dwell['Total']!='x']
df_dwell['Condominium']=pd.to_numeric(df_dwell['Condominium'])
df_dwell['Total']=pd.to_numeric(df_dwell['Total'])
#proportion of residents that live in condo
df_dwell['condo_dwelling']=(df_dwell['Condominium'])/(df_dwell['Total'])

In [ ]:
df_dwell=df_dwell.reset_index()
df_dwell.head()

Sub Category,Geo Code,Total,Condominium,Not condominium,condo_dwelling
0,Canada,14072080,1874835,12197240,0.133231
1,M1B,20235,5570,14660,0.275266
2,M1C,11270,300,10970,0.026619
3,M1G,9825,295,9535,0.030025
4,M1H,8930,3310,5625,0.370661


In [ ]:
df_dwell=df_dwell.drop(columns=['Total','Condominium','Not condominium'], axis=1)

## Age and Gender

In [ ]:
#load age/gender by FSA
other_census=pd.read_csv('Census_age.csv', dtype={'FSA':object, 
                                                 'Average_Age': np.float32,
                                                 'Median_Age': np.float32,
                                                 'Percent_male': np.float32})

In [ ]:
#Merge Demographic data
from functools import reduce
data_frames=[population_density_by_FSA,df_condition, df_dwell]
df_merge=reduce(lambda left, right: pd.merge(left, right, on=['Geo Code'], how='outer'),data_frames)

In [ ]:
df_merge=df_merge.rename(columns={'Geo Code': 'FSA'})

In [ ]:
data_frames=[df_merge, other_census]
df_demographics=reduce(lambda left, right: pd.merge(left, right, on=['FSA'], how='outer'), data_frames)

In [ ]:
df_demographics.to_csv('demographics.csv')